In [0]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, ActivityRegularization, Lambda
from keras.layers import Convolution2D, MaxPooling2D, Conv1D, MaxPooling1D, TimeDistributed
from keras.layers import AveragePooling2D, Input
from keras.utils import np_utils, normalize
from keras.engine import InputLayer
from keras import backend as K

import os
import scipy.io as sio
import h5py
import glob
import time

import sklearn
from sklearn.preprocessing import normalize

from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
from sklearn.linear_model import SGDClassifier


import matplotlib.pyplot as plt
import numpy as np
import time

Using TensorFlow backend.


In [0]:
network = VGG19(weights='imagenet')#, input_shape=(fscale, fscale, 3))
print(network.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
bloc

In [0]:
#dataset's path 
dir_path_DataSet = 'drive/My Drive/medical_image_recognition/datasets/'

#path of dataSet's fold 
save_fold='datanp_vgg/'

In [0]:
def read_dir(dir_path) :
  """return the differents categories, the images in these categories, and the number of images per category"""
  listDir = sorted(os.listdir(dir_path))
  sizes = []
  listFiles = []
  for d in listDir :
    files = sorted(os.listdir(dir_path+'/'+d))
    sizes += [len(files)]
    listFiles += [files]
  return listDir, listFiles, sizes


def create_zero_array(layer, n, nb_images):
  """return a numpy array with zeros and with the needed scales"""
  

  if layer == 'block5_pool' :
    X = np.zeros((nb_images, 7 + 10*n, 7 + 10*n, 512),dtype=np.float16)
    
  else :
    if n == 0 :
      X = np.zeros((nb_images, 4096),dtype=np.float16)
    else :
#       X = np.zeros((nb_images, -3 + 5*n, -3 + 5*n, 4096),dtype=np.float16) #???
        X = np.zeros((nb_images, 1 + 10*n, 1 + 10*n, 4096),dtype=np.float16)
  return X
  

def complete_array(network2,dir_path, dir_save, listDir, listFiles, X_new, n, net3) :
  """for all the images in listFiles, extract the features thanks to the network"""
  """return a 4-dimensional array with a 3-dimensional array per image"""
  nb = 0
  cpt = 0
  t0 = time.time()
  for i in range (len(listDir)) :
    d = listDir[i]
    files = listFiles[i]
    print(d, len(files))
    nb += len(files)
    for f in files :
#       print(str(f))
      t1 = time.time()
      img = image.load_img(dir_path+'/'+d+'/'+f, target_size=(224 + 320*n, 224 + 320*n))
      x = image.img_to_array(img)
      x = np.expand_dims(x, axis=0)
#       print(x.shape)
      x = preprocess_input(x)
      
      temp = net3.predict(x)
#       print( temp.shape)
      
      features = network2.predict(x)
#       print(features.shape)

      X_new[cpt,:] = features[0,:]
#       print(features.shape)
#       input()
      cpt += 1
      t2 = time.time()
      #print(str(cpt) + '/' + str(nb), t2 - t1)
  #np.save(dir_save, X_new)
  print('total', t2 - t0)
  return X_new

def pool(X, p):
  """return the numpy array X on which the pooling p was performed"""
  if(len(X.shape) == 2):
    return X
  if p == 'max' :  
    X_pool = np.max(X, axis = (1, 2)) 
  elif p == 'mean' :
    X_pool = np.mean(X, axis = (1, 2)) 
  else :
    X_pool = X
  return X_pool


def to_fully_conv(model):
  """transforms the Convolutional Neural Network model into a Fully Convolutional Network"""

  new_model = Sequential()

  input_layer = InputLayer(input_shape=(None, None, 3), name="input_new")

  new_model.add(input_layer)

  for layer in model.layers:

      if "Flatten" in str(layer):
          flattened_ipt = True
          f_dim = layer.input_shape
          print("get into flatten")
          continue

      elif "Dense" in str(layer):

          input_shape = layer.input_shape
          output_dim =  layer.get_weights()[1].shape[0]
          W,b = layer.get_weights()

          if flattened_ipt:
              shape = (f_dim[1],f_dim[2],f_dim[3],output_dim)
              new_W = W.reshape(shape)

              aa = np.array([new_W,b])
#               print(aa.shape)
              new_layer = Convolution2D(output_dim,
                                        (f_dim[1],f_dim[2]),
                                        #strides=(1,1),
                                        activation=layer.activation,
                                        padding='valid',
                                        weights=[new_W,b])
              flattened_ipt = False

          else:
              shape = (1,1,input_shape[1],output_dim)
              new_W = W.reshape(shape)

              new_layer = Convolution2D(output_dim,
                                        (1,1),
                                        #strides=(1,1),
                                        activation=layer.activation,
                                        padding='valid',
                                        weights=[new_W,b])


      else:
          new_layer = layer
      new_model.add(new_layer)

  return new_model

In [0]:
#name of datasets
# docs=['chest_xray','miniMIT_Etus','kvasir-dataset-v2']
docs=['miniMIT_Etus']
#images scales
scales=[1,2, 0]
#layer's names
layers=['fc1','fc2','block5_pool']
# pooling = 'mean'


for layer in layers:
  #definition of the network used (VGG19)
  network = VGG19(weights='imagenet')#, input_shape=(fscale, fscale, 3))
  net2 = Model(inputs=network.input, outputs=network.get_layer('block5_pool').output)

  network3 = Model(inputs=network.input, outputs=network.get_layer(layer).output)
  for doc in docs:
    for n in scales:
    
      if n >= 1 : 
        network2 = to_fully_conv(network3)
        net3 = to_fully_conv(net2)
        
      else:
        network2=network3
        net3 = net2 #??????
      
      
        
      print( "doc = " + doc + "layer = " + layer + " n = " + str(n) )
      if doc == 'kvasir-dataset-v2':
        #path where all images are stocked
        dir_path_X =  dir_path_DataSet + doc 
        #path where the output array will be saanpved
        path_save_X = dir_path_DataSet + doc + '/' + save_fold + doc + 'X_'+layer  +'(' + str(n) + ').npy'
        dir_save_Y  = dir_path_DataSet + doc + '/' + save_fold + doc + 'Y_'+layer  +'(' + str(n) + ').npy'

        #creating numpy array from images
        listDir_X, listFiles_X, sizes_X = read_dir(dir_path_X)
        X = create_zero_array(layer, n, sum(sizes_X))
        X = complete_array(network2,dir_path_X, path_save_X, listDir_X, listFiles_X, X, n)
        # create Y and save it
        Y = np.zeros(shape=(np.sum(sizes_X)))  

        init_size=0
        for i in range (len(listDir_X)):
          size=sizes_X[i]
          Y[init_size:init_size+size]=i
          init_size+=size 
          
        x_train,x_test,y_train,y_test=train_test_split(X, Y, test_size=0.2, random_state=42)


      else:
        #path where train images and test images are stocked
        dir_path_train =  dir_path_DataSet + doc + '/train'
        dir_path_test  =  dir_path_DataSet + doc + '/test'

        #path where the output array will be saved
        path_save_train = dir_path_DataSet + doc + '/' + save_fold + doc + 'X_train_'+ layer  +'(' + str(n) + ').npy'
        path_save_test  = dir_path_DataSet + doc + '/' + save_fold + doc + 'X_test_' + layer   +'(' + str(n) + ').npy'

        #creating numpy array from images
        listDir_train, listFiles_train, sizes_train = read_dir(dir_path_train)
        listDir_test, listFiles_test, sizes_test = read_dir(dir_path_test)


        X_train = create_zero_array(layer, n, sum(sizes_train))
        X_test = create_zero_array(layer, n, sum(sizes_test))
        
        X_train = complete_array(network2,dir_path_train, path_save_train, listDir_train, listFiles_train, X_train, n,net3)        
        X_test = complete_array(network2,dir_path_test, path_save_test, listDir_test, listFiles_test, X_test, n, net3)
        
# #         pooling
#         X_train = pool(X_train, pooling)
#         X_test = pool(X_test, pooling)
        
        dir_save_Y_train = dir_path_DataSet + doc + '/' + save_fold + doc + 'Y_train_'+ layer +'(' + str(n) + ').npy'
        dir_save_Y_test  = dir_path_DataSet + doc + '/' + save_fold + doc + 'Y_test_' + layer +'(' + str(n) + ').npy'
    
        # create Y_train and save it
        Y_train = np.zeros(shape=(np.sum(sizes_train)))  

        init_size=0
        for i in range (len(listDir_train)):
          size=sizes_train[i]
          Y_train[init_size:init_size+size]=i
          init_size+=size

        # create Y_test and save it
        Y_test = np.zeros(shape=(np.sum(sizes_test)))  

        init_size=0
        for i in range (len(listDir_test)):
          size=sizes_test[i]
          Y_test[init_size:init_size+size]=i
          init_size+=size


      #save data 
      np.save(dir_save_Y_train, Y_train)
      np.save(dir_save_Y_test, Y_test)
      #save data 
      np.save(path_save_train, X_train)
      np.save(path_save_test, X_test)
  
      
  
  

get into flatten
doc = miniMIT_Etuslayer = fc1 n = 1
bookstore 40
inside_bus 40
library 40
total 47.59290313720703
bookstore 40
inside_bus 40
library 40
total 47.09196877479553
get into flatten
doc = miniMIT_Etuslayer = fc1 n = 2
bookstore 40
inside_bus 40
library 40
total 102.76506662368774
bookstore 40
inside_bus 40
library 40
total 102.1616678237915
doc = miniMIT_Etuslayer = fc1 n = 0
bookstore 40
inside_bus 40
library 40
total 9.5138578414917
bookstore 40
inside_bus 40
library 40
total 8.39560341835022
get into flatten
doc = miniMIT_Etuslayer = fc2 n = 1
bookstore 40
inside_bus 40
library 40
total 48.705116987228394
bookstore 40
inside_bus 40
library 40
total 48.10479474067688
get into flatten
doc = miniMIT_Etuslayer = fc2 n = 2
bookstore 40
inside_bus 40
library 40
total 104.30459022521973
bookstore 40
inside_bus 40
library 40
total 103.65865993499756
doc = miniMIT_Etuslayer = fc2 n = 0
bookstore 40
inside_bus 40
library 40
total 9.368441343307495
bookstore 40
inside_bus 40
librar

In [0]:
X_train.shape

(120, 7, 7, 512)